Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/keras/train-hyperparameter-tune-deploy-with-keras/train-hyperparameter-tune-deploy-with-keras.png)

Let's get started. First let's import some Python libraries.

In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
import librosa
import librosa.display
from path import Path

In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.20.0


## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: aiforgoodgp1
Azure region: eastus
Subscription id: 7f1957e6-ab9f-44e4-82eb-9e4b6cb8a45e
Resource group: project15group1


Use the `register()` method to register datasets to your workspace so they can be shared with others, reused across various experiments, and referred to by name in your training script.
You can try get the dataset first to see if it's already registered.

## Register the model



We can then register the folder (and all files in it) as a model named `aiforgoodp15` under the workspace for deployment.

In [ ]:
from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "./output",
                       model_name = "aiforgoodp15",
                       description = "AI for good",
                       workspace = ws)

## Deploy the model in ACI
Now we are ready to deploy the model as a web service running in Azure Container Instance [ACI](https://azure.microsoft.com/en-us/services/container-instances/). Azure Machine Learning accomplishes this by constructing a Docker image with the scoring logic and model baked in.
### Create score.py
First, we will create a scoring script that will be invoked by the web service call. 

* Note that the scoring script must have two required functions, `init()` and `run(input_data)`. 
  * In `init()` function, you typically load the model into a global object. This function is executed only once when the Docker container is started. 
  * In `run(input_data)` function, the model is used to predict a value based on the input data. The input and output to `run` typically use JSON as serialization and de-serialization format but you are not limited to that.

In [109]:
import json
import numpy as np
import os
from keras.models import model_from_json
from tensorflow import keras
from azureml.core.model import Model

def init():
  
    global model
    global train_mean
    global classes

    classes= {1: 'car_horn', 3: 'dog_bark', 2: 'children_playing', 
          0: 'air_conditioner', 5: 'engine_idling', 7: 'jackhammer', 
          6: 'gun_shot', 9: 'street_music', 4: 'drilling', 8: 'siren', 10: 'Elephant'}
    
    #Get the path to where the model is stored in AML
    model_1_path = Model.get_model_path('aiforgoodp15')

    #Build out the complete path and file name
    modelname  = '/'+'spectrogram_save.h5'
    model_complete = model_1_path + modelname

    #Load the model from AML model registry
    model = keras.models.load_model(model_complete,compile=False)
    #model = keras.models.load_model('./output/spectrogram_save.h5')

    stored_array_name = '/'+'file.npz'
    stored_array_complete = model_1_path + stored_array_name

    loaded = np.load(stored_array_complete)
    #loaded = np.load('./output/file.npz')
    train_mean = loaded['train_mean']
    
def run(audio_data):

    filename,name = audio_data,audio_data.split('/')[-1]
    
    #Create the spectrogram
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('./' + name + '.jpg')
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    
        
    #Convert the spectrogram to a format for the model can use
    test= np.array(glob(filename))
    img_size=32
    Input = np.stack([np.asarray(Image.open(l).convert('RGB').resize((img_size, img_size))) for l in test], axis=0)
    
    ##Predict the class using the model
    output = model.predict((Input - train_mean[None,None,None,:])/256.)
    preds = [np.argmax(l) for l in output]
    cls_preds = [classes[l] for l in preds]
    
    #Return the results of the prediction 
    return (preds,cls_preds)

In [80]:
init()

In [89]:
run('./nn01a_20180203_1230-Copy1.wav')

([6], ['gun_shot'])

In [82]:
%%writefile score.py
import json
import numpy as np
import os
from keras.models import model_from_json
from tensorflow import keras
from azureml.core.model import Model

def init():
  
    global model
    global train_mean
    global classes

    classes= {1: 'car_horn', 3: 'dog_bark', 2: 'children_playing', 
          0: 'air_conditioner', 5: 'engine_idling', 7: 'jackhammer', 
          6: 'gun_shot', 9: 'street_music', 4: 'drilling', 8: 'siren', 10: 'Elephant'}
    
    #Get the path to where the model is stored in AML
    model_1_path = Model.get_model_path('aiforgoodp15')

    #Build out the complete path and file name
    modelname  = '/'+'spectrogram_save.h5'
    model_complete = model_1_path + modelname

    #Load the model from AML model registry
    model = keras.models.load_model(model_complete,compile=False)
    #model = keras.models.load_model('./output/spectrogram_save.h5')

    stored_array_name = '/'+'file.npz'
    stored_array_complete = model_1_path + stored_array_name

    loaded = np.load(stored_array_complete)
    #loaded = np.load('./output/file.npz')
    train_mean = loaded['train_mean']
    
def run(audio_data):

    filename,name = audio_data,audio_data.split('/')[-1]
    
    #Create the spectrogram
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('./' + name + '.jpg')
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    
        
    #Convert the spectrogram to a format for the model can use
    test= np.array(glob(filename))
    img_size=32
    Input = np.stack([np.asarray(Image.open(l).convert('RGB').resize((img_size, img_size))) for l in test], axis=0)
    
    ##Predict the class using the model
    output = model.predict((Input - train_mean[None,None,None,:])/256.)
    preds = [np.argmax(l) for l in output]
    cls_preds = [classes[l] for l in preds]
    
    #Return the results of the prediction 
    return (preds,cls_preds)

Overwriting score.py


### Create myenv.yml
We also need to create an environment file so that Azure Machine Learning can install the necessary packages in the Docker image which are required by your scoring script. In this case, we need to specify conda packages `tensorflow` and `keras`.

In [17]:
!pip freeze

absl-py==0.11.0
adal==1.2.5
aiohttp==3.7.3
aiohttp-cors==0.7.0
aioredis==1.3.1
alembic==1.4.1
ansiwrap==0.8.4
antlr4-python3-runtime==4.7.2
appdirs==1.4.4
applicationinsights==0.11.9
argcomplete==1.12.2
argon2-cffi==20.1.0
astor==0.8.1
astroid==2.4.2
astunparse==1.6.3
async-timeout==3.0.1
atari-py==0.2.6
attrs==20.3.0
audioread==2.1.9
autopep8==1.5.4
azure-appconfiguration==1.1.1
azure-batch==10.0.0
azure-cli==2.17.1
azure-cli-core==2.17.1
azure-cli-telemetry==1.0.6
azure-common==1.1.26
azure-core==1.9.0
azure-cosmos==3.2.0
azure-datalake-store==0.0.51
azure-functions-devops-build==0.0.22
azure-graphrbac==0.61.1
azure-identity==1.4.1
azure-keyvault==1.1.0
azure-keyvault-administration==4.0.0b2
azure-loganalytics==0.1.0
azure-mgmt-advisor==2.0.1
azure-mgmt-apimanagement==0.2.0
azure-mgmt-appconfiguration==0.6.0
azure-mgmt-applicationinsights==0.1.1
azure-mgmt-authorization==0.61.0
azure-mgmt-batch==9.0.0
azure-mgmt-batchai==2.0.0
azure-mgmt-billing==1.0.0
azure-mgmt-botservice==0.2.0
az

In [83]:
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('h5py<=2.10.0')
cd.add_conda_package('keras<=2.3.1')
cd.add_conda_package('Pillow==6.2.1')
cd.add_conda_package('librosa==0.8.0')
cd.add_conda_package('matplotlib==3.2.1')
cd.add_pip_package('tensorflow==2.4.0')
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')
print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults~=1.20.0
  - tensorflow==2.4.0
- h5py<=2.10.0
- keras<=2.3.1
- Pillow==6.2.1
- librosa==0.8.0
- matplotlib==3.2.1
channels:
- anaconda
- conda-forge



### Deploy to ACI
We are almost ready to deploy. Create the inference configuration and deployment configuration and deploy to ACI. This cell will run for about 7-8 minutes.

In [84]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
myenv = myenv.register(ws)
bld = myenv.build(ws)
bld.wait_for_completion(show_output=True)


Image Build Status: Running

2021/02/23 22:37:10 Downloading source code...
2021/02/23 22:37:12 Finished downloading source code
2021/02/23 22:37:12 Creating Docker network: acb_default_network, driver: 'bridge'
2021/02/23 22:37:13 Successfully set up Docker network: acb_default_network
2021/02/23 22:37:13 Setting up Docker configuration...
2021/02/23 22:37:14 Successfully set up Docker configuration
2021/02/23 22:37:14 Logging in to registry: aiforgoodgp1.azurecr.io
2021/02/23 22:37:15 Successfully logged into aiforgoodgp1.azurecr.io
2021/02/23 22:37:15 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/02/23 22:37:15 Scanning for dependencies...
2021/02/23 22:37:16 Successfully scanned dependencies
2021/02/23 22:37:16 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB

Step 1/18 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1@sha256:81ca5cdfeefc2a25afff41e66151e97


pillow-6.2.1         | 640 KB    |            |   0% 
pillow-6.2.1         | 640 KB    | ########## | 100% 

threadpoolctl-2.1.0  | 16 KB     |            |   0% 
threadpoolctl-2.1.0  | 16 KB     | ########## | 100% 

libedit-3.1          | 171 KB    |            |   0% 
libedit-3.1          | 171 KB    | ########## | 100% 

tk-8.6.10            | 3.2 MB    |            |   0% 
tk-8.6.10            | 3.2 MB    | ########## | 100% 

matplotlib-base-3.2. | 7.1 MB    |            |   0% 
matplotlib-base-3.2. | 7.1 MB    | ####3      |  44% 
matplotlib-base-3.2. | 7.1 MB    | ########## | 100% 

astor-0.8.1          | 45 KB     |            |   0% 
astor-0.8.1          | 45 KB     | ########## | 100% 

scikit-learn-0.23.2  | 6.9 MB    |            |   0% 
scikit-learn-0.23.2  | 6.9 MB    | ######3    |  63% 
scikit-learn-0.23.2  | 6.9 MB    | ########## | 100% 

certifi-2020.6.20    | 160 KB    |            |   0% 
certifi-2020.6.20    | 160 KB    | ########## | 100% 

werkzeug-0.16.1    


tensorflow-base-2.0. | 100.9 MB  | ########## | 100% 

libvpx-1.7.0         | 2.4 MB    |            |   0% 
libvpx-1.7.0         | 2.4 MB    | ########## | 100% 

absl-py-0.10.0       | 168 KB    |            |   0% 
absl-py-0.10.0       | 168 KB    | ########## | 100% 

x264-1!157.20191217  | 2.5 MB    |            |   0% 
x264-1!157.20191217  | 2.5 MB    | ########## | 100% 

libstdcxx-ng-9.1.0   | 4.0 MB    |            |   0% 
libstdcxx-ng-9.1.0   | 4.0 MB    |            |   0% 
libstdcxx-ng-9.1.0   | 4.0 MB    | #########3 |  94% 
libstdcxx-ng-9.1.0   | 4.0 MB    | ########## | 100% 

decorator-4.4.2      | 14 KB     |            |   0% 
decorator-4.4.2      | 14 KB     | ########## | 100% 

blas-1.0             | 6 KB      |            |   0% 
blas-1.0             | 6 KB      | ########## | 100% 

appdirs-1.4.4        | 13 KB     |            |   0% 
appdirs-1.4.4        | 13 KB     | ########## | 100% 

pysoundfile-0.10.3.p | 23 KB     |            |   0% 
pysoundfile-0.10.3.


#
# To activate this environment, use
#
#     $ conda activate /azureml-envs/azureml_d66b695a90f0d7c9f18c84a75fb1fbb0
#
# To deactivate an active environment, use
#
#     $ conda deactivate



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda


Removing intermediate container 1c6cf3d53c82
 ---> fd3a44fa46fe
Step 9/18 : ENV PATH /azureml-envs/azureml_d66b695a90f0d7c9f18c84a75fb1fbb0/bin:$PATH
 ---> Running in 6a613b458b85
Removing intermediate container 6a613b458b85
 ---> 5ce32ead4574
Step 10/18 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> da0d11c0f3ae
Step 11/18 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> 3fd75f406b58
Step 12/18 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azure

In [87]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2,
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=2,
                                               tags={'name': 'aiforgoodp15', 'framework': 'Keras'},
                                               description='Keras MLP on wav')

registered_model = Model(ws,'aiforgoodp15')

service = Model.deploy(workspace=ws, 
                           name='aiforgoodp15v6', 
                           models=[registered_model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


**Tip: If something goes wrong with the deployment, the first thing to look at is the logs from the service by running the following command:**

In [ ]:
print(service.get_logs())

This is the scoring web service endpoint:

In [92]:
print(service.scoring_uri)

http://6c9792fc-56c3-4631-8fa1-35013661721e.eastus.azurecontainer.io/score


### Test the deployed model
Let's test the deployed model. Pick 30 random samples from the test set, and send it to the web service hosted in ACI. Note here we are using the `run` API in the SDK to invoke the service. You can also make raw HTTP calls using any HTTP tool such as curl.

After the invocation, we print the returned predictions and plot them along with the input images. Use red font color and inversed image (white on black) to highlight the misclassified samples. Note since the model accuracy is pretty high, you might have to run the below cell a few times before you can see a misclassified sample.

We can retrieve the API keys used for accessing the HTTP endpoint.

In [93]:
# Retrieve the API keys. Two keys were generated.
key1, Key2 = service.get_keys()
print(key1)

mwBrQDOExB6buHYjXHesgr1NRIvkBv4p


We can now send construct raw HTTP request and send to the service. Don't forget to add key to the HTTP header.

In [107]:
import requests

audio_data    = './nn01a_20180203_1230-Copy1.wav'
print(type(audio_data))
headers = {'Content-Type':'audio/wav','Content-Disposition':'attachment; filename=audio_data ', 'Authorization': 'Bearer ' + key1}
resp = requests.post(service.scoring_uri, headers=headers)
resp = requests.post(service.scoring_uri, audio_data, headers=headers)

print("POST to url", service.scoring_uri)
print(resp)


<class 'str'>
POST to url http://6c9792fc-56c3-4631-8fa1-35013661721e.eastus.azurecontainer.io/score
<Response [415]>


Let's look at the workspace after the web service was deployed. You should see 
* a registered model named 'keras-mlp-mnist' and with the id 'model:1'  
* a webservice called 'keras-mnist-svc' with some scoring URL

In [ ]:
model = ws.models['keras-mlp-mnist']
print("Model: {}, ID: {}".format('keras-mlp-mnist', model.id))
    
webservice = ws.webservices['keras-mnist-svc']
print("Webservice: {}, scoring URI: {}".format('keras-mnist-svc', webservice.scoring_uri))

## Clean up
You can delete the ACI deployment with a simple delete API call.

In [ ]:
service.delete()